# Select real barcode spots

The base calling is done on everything that could be a rabies spot. We then need to
select the real spots before doing error correction.

In [ ]:
%load_ext autoreload
%autoreload 2

## Load an example tile

In [ ]:
import numpy as np
import pandas as pd
import iss_preprocess as issp
from iss_preprocess.vis import plot_matrix_with_colorbar
from iss_preprocess.pipeline.sequencing import load_spot_sign_image
import matplotlib.pyplot as plt

mouse = "BRAC8498.3e"
chamber = "chamber_07"
data_path = f"becalia_rabies_barseq/{mouse}/{chamber}"
ref_tile_index = 0  # which of the reference tiles do we want to use for plots

In [ ]:
# Load some example data
processed_path = issp.io.get_processed_path(data_path)
ops = issp.io.load.load_ops(data_path)
tile_coors = ops["barcode_ref_tiles"][ref_tile_index]
cluster_means = np.load(processed_path / "barcode_cluster_means.npy")

stack, bad_pixels = issp.pipeline.load_and_register_sequencing_tile(
    data_path,
    tile_coors,
    filter_r=ops["filter_r"],
    prefix="barcode_round",
    suffix=ops["barcode_projection"],
    nrounds=ops["barcode_rounds"],
    correct_channels=ops["barcode_correct_channels"],
    corrected_shifts=ops["corrected_shifts"],
    correct_illumination=True,
)
stack = stack[:, :, np.argsort(ops["camera_order"]), :]
stack[bad_pixels, :, :] = 0
ref = stack.std(axis=(2, 3))
print(stack.shape)

In [ ]:
ops = issp.io.load.load_ops(data_path)
data_folder = issp.io.get_processed_path(data_path)
roi = tile_coors[0]
barcode_spots = pd.read_pickle(
    data_folder / f"barcode_round_spots_{roi}.pkl"
)
tile_name = "_".join([str(t) for t in tile_coors])
barcode_spots.query("tile == @tile_name", inplace=True)
print(barcode_spots.shape)
tile_corners = issp.pipeline.stitch.get_tile_corners(data_path, prefix=ops['reference_prefix'], roi=tile_coors[0])
tile_origin = tile_corners[tile_coors[1], tile_coors[2]].min(axis=1)


In [ ]:
barcode_spots["x_tile"] = barcode_spots["x"] - tile_origin[1]
barcode_spots["y_tile"] = barcode_spots["y"] - tile_origin[0]
ax = plt.subplot(1,1,1)
ax.set_title(f"Tile {tile_coors} - all spots")
cax, cb = plot_matrix_with_colorbar(ref, ax=ax,vmax=0.1, interpolation="none")
cb.set_label("Std")
ax.scatter(barcode_spots["x_tile"], barcode_spots["y_tile"], s=1, c="r", alpha=0.1)
center = np.array([700, 2100])
ax.scatter(center[1], center[0], s=10, c="green", alpha=1)


# Show various thresholds

In [ ]:
thresholds=dict(mean_intensity_threshold=0.03,
dot_product_score_threshold=0.2,
mean_score_threshold=0.9
)

In [ ]:
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
fig = plt.figure(figsize=(15, 5))

win = 150
for iax, (prop_th, thres) in enumerate(thresholds.items()):
    prop = prop_th.replace("_threshold", "")
    ax = plt.subplot(1, 3, 1 + iax)
    ax.set_title(f"{prop}")
    ax.imshow(ref, vmax=0.2, interpolation="none", cmap="gray")
    ax.set_xticks([])
    ax.set_yticks([])
    ax_divider = make_axes_locatable(ax)
    cax = ax_divider.append_axes("right", size="7%", pad="2%")
    vmin, vmax = thres * np.array([1/ 2, 2])
    if prop == "mean_score":
        vmin, vmax = [0.8, 1]
    
    sc = ax.scatter(barcode_spots["x_tile"], barcode_spots["y_tile"], s=5, c=barcode_spots[prop], vmin=vmin, vmax=vmax, alpha=1)
    cb = ax.figure.colorbar(sc, cax=cax)
    cax.axhline(thres, color="r")
    ax.set_xlim(center[1] - win, center[1] + win)
    ax.set_ylim(center[0] + win, center[0] - win)

fig.tight_layout()

In [ ]:
# Plot all rounds
window = 150
lims = np.vstack([center - window, center + window]).astype(int)
lims = np.clip(lims, 0, np.array(stack.shape[:2]) - 1)
nr = ops["barcode_rounds"]
stack_part = stack[lims[0, 0] : lims[1, 0], lims[0, 1] : lims[1, 1], :]
valid_spots = barcode_spots[
    (barcode_spots.x_tile > lims[0, 1])
    & (barcode_spots.x_tile < lims[1, 1])
    & (barcode_spots.y_tile > lims[0, 0])
    & (barcode_spots.y_tile < lims[1, 0])
]
print(f"Valid spots: {valid_spots.shape[0]}")
for prop_th, th in thresholds.items():
    prop = prop_th.replace("_threshold", "")
    valid_spots = valid_spots.query(f"{prop} > {th}")
    print(f"Valid spots after {prop}: {valid_spots.shape[0]}")


fig = plt.figure(figsize=(15, 10))
channel_colors = ([1, 0, 0], [0, 1, 0], [1, 0, 1], [0, 1, 1])
nr = 14
for iround in range(nr):
    ax = plt.subplot(3, 5, 1 + iround)
    ax.set_title(f"Round {iround}")
    rgb_stack = issp.vis.round_to_rgb(
                stack_part, iround, extent=None, channel_colors=channel_colors
            )
    # plot raw fluo
    ax.imshow(rgb_stack)
    
    ax.scatter(valid_spots["x_tile"] - lims[0, 1] +6, valid_spots["y_tile"] - lims[0, 0]-4, s=50,
               marker='o', edgecolors='w', facecolors='none', alpha=1)
    ax.set_xticks([])
    ax.set_yticks([])
fig.tight_layout()

# GMM filter the rest


Do that on all chambers

In [ ]:
import iss_analysis as issa

valid_components = [1]
data_path = f"becalia_rabies_barseq/{mouse}/"

barcode_spots, gmm, all_barcode_spots = issa.barcodes.barcodes.get_barcodes(
    acquisition_folder=data_path,
    n_components=3,
    valid_components=valid_components,
    **thresholds,
)


In [ ]:
from iss_analysis.barcodes.diagnostics import plot_gmm_clusters
pg = plot_gmm_clusters(all_barcode_spots, gmm, thresholds=thresholds)
fig = pg.figure


In [ ]:
ch_r_gp = all_barcode_spots.groupby(["chamber", "roi"])
n_rois = len(ch_r_gp)
n_comp = len(all_barcode_spots.gmm_label.unique())
n_per_lab = np.zeros((n_rois, n_comp))

for i, (ch_r, df) in enumerate(ch_r_gp):
    n_per_lab[i] = df.groupby("gmm_label").size()

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(2, 1, 1)
ax.plot(np.arange(n_rois), n_per_lab, "o-")
ax.plot(np.arange(n_rois), np.sum(n_per_lab, axis=1), "o-", color="black")
ax.set_xticks(np.arange(n_rois))
ax.set_xticklabels([])
ax.set_ylabel("Number of spots")
ax = fig.add_subplot(2, 1, 2)
ax.plot(np.arange(n_rois), n_per_lab / np.sum(n_per_lab, axis=1, keepdims=True), "o-")
ax.set_xticks(np.arange(n_rois))
ax.set_ylabel("Fraction of spots")
_ = ax.set_xticklabels([f"{ch}_{r}" for ch, r in ch_r_gp.groups.keys()], rotation=90)
fig.tight_layout()

In [ ]:
ex_chamber = 9
roi = 9

import matplotlib.pyplot as plt

df = all_barcode_spots[
    (all_barcode_spots["chamber"] == f"chamber_{ex_chamber:02}")
    & (all_barcode_spots["roi"] == roi)
]

for i in range(n_comp):
    fig = plt.figure(figsize=(10, 10), facecolor="w")
    ax = fig.add_subplot(1, 1, 1, aspect="equal", facecolor="w")
    ax.scatter(
        df[df["gmm_label"] == i]["y"],
        df[df["gmm_label"] == i]["x"],
        label=f"cluster {i}",
        alpha=0.2,
        s=3,
        color=f"k",
    )
    ax.set_xlim(5000, 15000)
    ax.set_ylim(10000, 18000)
    # _ = ax.axis("off")
    ax.legend()
    break
fig.tight_layout()


In [ ]:
chamber, roi

In [ ]:
# Plot all rounds

barcode_spots = all_barcode_spots.query("chamber == @chamber")
barcode_spots.query("tile == @tile_name", inplace=True)
barcode_spots["x_tile"] = barcode_spots["x"] - tile_origin[1]
barcode_spots["y_tile"] = barcode_spots["y"] - tile_origin[0]

window = 100
lims = np.vstack([center - window, center + window]).astype(int)
lims = np.clip(lims, 0, np.array(stack.shape[:2]) - 1)
nr = ops["barcode_rounds"]
stack_part = stack[lims[0, 0] : lims[1, 0], lims[0, 1] : lims[1, 1], :]
valid_spots = barcode_spots[
    (barcode_spots.x_tile > lims[0, 1])
    & (barcode_spots.x_tile < lims[1, 1])
    & (barcode_spots.y_tile > lims[0, 0])
    & (barcode_spots.y_tile < lims[1, 0])
]
print(f"Valid spots: {valid_spots.shape[0]}")
for prop_th, th in thresholds.items():
    prop = prop_th.replace("_threshold", "")
    valid_spots = valid_spots.query(f"{prop} > {th}")
    print(f"Valid spots after {prop}: {valid_spots.shape[0]}")


fig = plt.figure(figsize=(15, 7))
channel_colors = ([1, 0, 0], [0, 1, 0], [1, 0, 1], [0, 1, 1])
nr = 10
for iround in range(nr):
    ax = plt.subplot(2, 5, 1 + iround)
    ax.set_title(f"Round {iround}")
    rgb_stack = issp.vis.round_to_rgb(
                stack_part, iround, extent=None, channel_colors=channel_colors
            )
    # plot raw fluo
    ax.imshow(rgb_stack)
    for lab, spots in valid_spots.groupby("gmm_label"):
        ax.scatter(spots["x_tile"] - lims[0, 1] +6, spots["y_tile"] - lims[0, 0]-4, s=50,
                   marker='o', facecolors='none', edgecolors=f'C{lab}', alpha=1, label=f"Cluster {lab}")
   
    ax.set_xticks([])
    ax.set_yticks([])
fig.tight_layout()
ax.legend()